# Normalization

Normalization is a crucial process for ensuring optimal performance and efficient management of data in file-based database systems like ParquetDB. In traditional databases, normalization typically refers to structuring relational tables to reduce redundancy. In ParquetDB’s context, **normalization** helps balance the distribution of data across multiple Parquet files, avoiding situations where files have uneven row counts. 

Without proper normalization, data skew can lead to performance bottlenecks in operations such as queries, inserts, updates, or deletions. By normalizing your dataset, ParquetDB rewrites and restructures your files to have a more consistent number of rows, improving parallelization and read/write speeds. 

In this notebook, we will:
1. Generate an example dataset using `generate_similar_data`.
2. Demonstrate how to normalize data in ParquetDB using `NormalizeConfig`.
3. Show how normalization can improve performance by ensuring each file has a balanced distribution of rows.

In [2]:
import pprint
import os
from pathlib import Path
import shutil
from parquetdb.utils.general_utils import generate_similar_data
from parquetdb import ParquetDB, NormalizeConfig

#### Generate Example Data

Below, we’ll generate a dataset that imitates real-world data variations using the `generate_similar_data` utility function. This function creates new data entries based on the structure of a provided template.


In [3]:
# Define a simple template data entry
template_dict = {
    "float_field": 10,
    "int_field": 10,
    "name": "item",
    "nested_value": {"value": 10, "name": "item"},
    "list_field": [1, 2, 3],
}
for x in range(500):
    template_dict[f"column_{x}"] = "test"

template = [template_dict]

# Generate multiple data entries
num_entries = 100000  # Feel free to adjust this
data = generate_similar_data(template, num_entries)

print("Generated Data:")
pprint.pprint(data[0])

Generated Data:
{'column_0': 'test_74',
 'column_1': 'test_16',
 'column_10': 'test_64',
 'column_100': 'test_84',
 'column_101': 'test_76',
 'column_102': 'test_78',
 'column_103': 'test_26',
 'column_104': 'test_16',
 'column_105': 'test_70',
 'column_106': 'test_19',
 'column_107': 'test_5',
 'column_108': 'test_82',
 'column_109': 'test_54',
 'column_11': 'test_72',
 'column_110': 'test_94',
 'column_111': 'test_59',
 'column_112': 'test_31',
 'column_113': 'test_33',
 'column_114': 'test_30',
 'column_115': 'test_82',
 'column_116': 'test_2',
 'column_117': 'test_20',
 'column_118': 'test_82',
 'column_119': 'test_31',
 'column_12': 'test_26',
 'column_120': 'test_89',
 'column_121': 'test_84',
 'column_122': 'test_85',
 'column_123': 'test_81',
 'column_124': 'test_37',
 'column_125': 'test_100',
 'column_126': 'test_95',
 'column_127': 'test_54',
 'column_128': 'test_86',
 'column_129': 'test_82',
 'column_13': 'test_5',
 'column_130': 'test_62',
 'column_131': 'test_94',
 'colu

Next, we import the data into our database


In [4]:
ROOT_DIR = Path(".")
DATA_DIR = ROOT_DIR / "data"

if DATA_DIR.exists():
    shutil.rmtree(DATA_DIR)
    
db_path = ROOT_DIR / "ParquetDB"

db = ParquetDB(db_path=db_path)

db.create(data)
print(db)

PARQUETDB SUMMARY
Database path: ParquetDB

• Number of columns: 507
• Number of rows: 100000
• Number of files: 1
• Number of rows per file: [100000]
• Number of row groups per file: [4]
• Serialized metadata size per file: [225147] Bytes

############################################################
METADATA
############################################################

############################################################
COLUMN DETAILS
############################################################
• Columns:
    - column_296
    - column_302
    - column_370
    - column_0
    - column_80
    - column_367
    - column_344
    - column_216
    - column_151
    - column_61
    - column_439
    - column_87
    - column_379
    - column_351
    - column_329
    - column_423
    - column_97
    - column_42
    - column_401
    - column_355
    - column_171
    - column_261
    - column_125
    - column_270
    - column_398
    - column_17
    - column_18
    - column_479
    - column

In [5]:
data = None
df = db.read().to_pandas()
print(df)

      column_296 column_302 column_370  column_0 column_80 column_367  \
0        test_33    test_30    test_29   test_74   test_59    test_43   
1        test_62    test_97    test_10   test_29   test_59    test_47   
2         test_6    test_65    test_28   test_29    test_6    test_81   
3        test_35    test_61    test_57   test_61   test_71    test_92   
4        test_56    test_44    test_70   test_18   test_75    test_77   
...          ...        ...        ...       ...       ...        ...   
99995    test_43    test_48    test_46   test_77   test_26    test_20   
99996     test_6    test_45    test_36  test_100   test_47    test_58   
99997     test_2    test_33    test_61   test_99   test_64    test_44   
99998    test_96    test_15    test_26   test_58   test_35    test_77   
99999    test_63    test_87    test_39   test_38   test_59    test_83   

      column_344 column_216 column_151 column_61  ... column_352 column_110  \
0        test_78    test_62    test_58   tes

## Normalize Data Using ParquetDB

Next, we'll introduce the `NormalizeConfig` class, which allows you to fine-tune how normalization is performed over the various operations in ParquetDB.

### The `NormalizeConfig` Class

```python 
@dataclass
class NormalizeConfig:
    load_format: str = "table"
    batch_size: int = 131_072
    batch_readahead: int = 16
    fragment_readahead: int = 4
    fragment_scan_options: Optional[pa.dataset.FragmentScanOptions] = None
    use_threads: bool = True
    memory_pool: Optional[pa.MemoryPool] = None
    filesystem: Optional[fs.FileSystem] = None
    file_options: Optional[ds.FileWriteOptions] = None
    use_threads: bool = config.parquetdb_config.normalize_kwargs.use_threads
    max_partitions: int = config.parquetdb_config.normalize_kwargs.max_partitions
    max_open_files: int = config.parquetdb_config.normalize_kwargs.max_open_files
    max_rows_per_file: int = config.parquetdb_config.normalize_kwargs.max_rows_per_file
    min_rows_per_group: int = (
        config.parquetdb_config.normalize_kwargs.min_rows_per_group
    )
    max_rows_per_group: int = (
        config.parquetdb_config.normalize_kwargs.max_rows_per_group
    )
    file_visitor: Optional[Callable] = None
    existing_data_behavior: str = (
        config.parquetdb_config.normalize_kwargs.existing_data_behavior
    )
    create_dir: bool = True

```

The `NormalizeConfig` data class allows you to fine-tune how normalization is performed. The most important parameters are the following:

- **`load_format : str`**  The format of the output dataset. Supported formats are `'table'` and `'batches'` (default: `'table'`).

- **`batch_size : int, optional`**  The number of rows to process in each batch (default: `None`).

- **`batch_readahead : int, optional`**  The number of batches to read ahead in a file (default: `16`).

- **`fragment_readahead : int, optional`**  The number of files to read ahead, improving IO utilization at the cost of RAM usage (default: `4`).

- **`max_open_files : int`**  Maximum open files for dataset writing (default: `1024`).

- **`max_rows_per_file : int`**  Maximum rows per file (default: `10,000`).

- **`min_rows_per_group : int`**  Minimum rows per row group within each file (default: `0`).

- **`max_rows_per_group : int`**  Maximum rows per row group within each file (default: `10,000`).


In parquet files, it stores the data in row groups, this allows for batching and parallelization. 

Below is a diagram of what your data will look like in memory. 

- Csv files use a row based system, which is inefficient as it does not store similar data contiguously in memory.
- Columnar storage is more efficient as it stores similar data contiguously in memory, however batching the data is not so great as data is not stored in chunks.
- Parquet files use a Row Group based system, which is more efficient as it stores similar data contiguously, but it also stores in chunks which is great for parallelization.

![Row Group Storage](../media/images/row_group_storage.png)


Optimizing parameters like the number of rows per row group, how many row groups per file, and how many files to read ahead can help significantly improve speed and memory performance.


Let's look at the details of the row groups of our current dataset. We can do this by using the `summary`

In [6]:
print(db.summary(show_row_group_metadata=True))

PARQUETDB SUMMARY
Database path: ParquetDB

• Number of columns: 507
• Number of rows: 100000
• Number of files: 1
• Number of rows per file: [100000]
• Number of row groups per file: [4]
• Number of rows per row group per file: 
    - ParquetDB_0.parquet:
        - Row group 0: 32768 rows
        - Row group 1: 32768 rows
        - Row group 2: 32768 rows
        - Row group 3: 1696 rows
• Serialized metadata size per file: [225147] Bytes

############################################################
METADATA
############################################################

############################################################
COLUMN DETAILS
############################################################



Here we can see that in our first file, we have 4 row groups where there is a maximum rows per group of 32,768. Typically, this is fine but if your system can handle it, it is best the chunk the data into larger groups.

A good rule of thumb for these settings should be about 2 GB per file and about 200MB per row group size. This will require some trial and error to find the best settings for your system.

Let's normalize the data with `NormalizeConfig` and change it so it is 50000 rows per row group.


In [25]:
from parquetdb import NormalizeConfig

normalize_config = NormalizeConfig(min_rows_per_group=50000, max_rows_per_group=50000)

db.normalize(normalize_config=normalize_config)

print(db.summary(show_row_group_metadata=True))

PARQUETDB SUMMARY
Database path: ParquetDB

• Number of columns: 507
• Number of rows: 100000
• Number of files: 1
• Number of rows per file: [100000]
• Number of row groups per file: [2]
• Number of rows per row group per file: 
    - ParquetDB_0.parquet:
        - Row group 0: 50000 rows
        - Row group 1: 50000 rows
• Serialized metadata size per file: [136357] Bytes

############################################################
METADATA
############################################################

############################################################
COLUMN DETAILS
############################################################



Now we have two row groups, each with a maximum of 50,000 rows. 

In some cases, however, this might be too large—especially if you're working with a wide dataset (e.g., ~4,000 columns). To handle such cases, we can reduce the number of rows per group to 10,000 for better performance.

For particularly large datasets, it's also important to fine-tune additional parameters, such as `batch_readahead`, `fragment_readahead`, `load_format="batches"`, and `batch_size`. 

By default, ParquetDB uses `load_format="table"` and `batch_size=None`, meaning it will attempt to write and read all the data at once. While this approach works well for smaller datasets, it can cause performance bottlenecks when handling larger datasets.

To address this, we can set `load_format="batches"` and define `batch_size=5000`. This configuration ensures that data is processed in chunks of 5,000 rows at a time, improving memory management. Additionally, setting `batch_readahead=2` allows ParquetDB to load two batches into memory ahead of processing, further enhancing performance by reducing waiting times.

When reading data, ParquetDB processes files sequentially. To optimize this process, we can control how many files are opened and read ahead by setting `fragment_readahead=2`. This ensures that the system reads two files ahead, balancing I/O performance and memory usage.

> Note: The batch size can only go as high as the number of rows in a row group.

In [26]:
normalize_config = NormalizeConfig(
    load_format="batches",
    batch_size=100,
    batch_readahead=16,
    fragment_readahead=4,
    max_rows_per_group=10000,
    min_rows_per_group=10000,
)

db.normalize(normalize_config=normalize_config)

print(db.summary(show_row_group_metadata=True))

PARQUETDB SUMMARY
Database path: ParquetDB

• Number of columns: 507
• Number of rows: 100000
• Number of files: 1
• Number of rows per file: [100000]
• Number of row groups per file: [10]
• Number of rows per row group per file: 
    - ParquetDB_0.parquet:
        - Row group 0: 10000 rows
        - Row group 1: 10000 rows
        - Row group 2: 10000 rows
        - Row group 3: 10000 rows
        - Row group 4: 10000 rows
        - Row group 5: 10000 rows
        - Row group 6: 10000 rows
        - Row group 7: 10000 rows
        - Row group 8: 10000 rows
        - Row group 9: 10000 rows
• Serialized metadata size per file: [497366] Bytes

############################################################
METADATA
############################################################

############################################################
COLUMN DETAILS
############################################################



### Finishing

Now that we normalized the data, we can see that the data is more evenly distributed across the row groups. Many methods, such as (`read`, `update`, `delete`, `transform`, `update_schema`), in ParquetDB take as an argument a `normalize_config` which allows you to fine-tune the normalization process during these operations.
